<a href="https://colab.research.google.com/github/ja390/Parallel-Computing-Assignment3/blob/main/Assignmet03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get update -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
!sudo apt-get install -y openmpi-bin libopenmpi-dev > /dev/null


In [3]:
print("✔ OpenMPI Installed")
!mpirun --version

✔ OpenMPI Installed
mpirun (Open MPI) 4.1.2

Report bugs to http://www.open-mpi.org/community/help/


In [4]:
print("\nChecking nvcc (CUDA compiler):")
!nvcc --version


Checking nvcc (CUDA compiler):
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
print("\nChecking GPU:")
!nvidia-smi


Checking GPU:
Wed Dec  3 04:05:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------

In [17]:
%%writefile kmeans_openmp.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>

#define MAX_ITER 100
#define K 3  // no of clusters

typedef struct {
    double x, y;
    int cluster;
} Point;

typedef struct {
    double x, y;
} Centroid;

// Distance function
double distance(Point a, Centroid b) {
    return sqrt((a.x - b.x)*(a.x - b.x) + (a.y - b.y)*(a.y - b.y));
}

int main(int argc, char *argv[]) {

    if (argc != 2) {
        printf("Usage: %s <num_threads>\n", argv[0]);
        return 1;
    }

    int num_threads = atoi(argv[1]);
    omp_set_num_threads(num_threads);

    int n = 9; // number of points
    Point points[9] = {
        {1,2}, {2,1}, {3,3},
        {8,8}, {9,8}, {8,9},
        {4,5}, {5,6}, {6,5}
    };

    Centroid centroids[K] = {{2,2}, {8,8}, {5,5}};

    double start = omp_get_wtime();

    for (int iter = 0; iter < MAX_ITER; iter++) {

        // parallel clusters
        #pragma omp parallel for
        for (int i = 0; i < n; i++) {
            double minDist = 1e9;
            int bestCluster = 0;

            for (int j = 0; j < K; j++) {
                double dist = distance(points[i], centroids[j]);
                if (dist < minDist) {
                    minDist = dist;
                    bestCluster = j;
                }
            }

            points[i].cluster = bestCluster;
        }

        //  Parallel centroid update  Using array reductions
        double sumX[K] = {0}, sumY[K] = {0};
        int count[K] = {0};

        #pragma omp parallel for reduction(+:sumX[:K], sumY[:K], count[:K])
        for (int i = 0; i < n; i++) {
            int c = points[i].cluster;
            sumX[c] += points[i].x;
            sumY[c] += points[i].y;
            count[c]++;
        }

        // Update centroids
        for (int j = 0; j < K; j++) {
            if (count[j] > 0) {
                centroids[j].x = sumX[j] / count[j];
                centroids[j].y = sumY[j] / count[j];
            }
        }
    }

    double end = omp_get_wtime();

    printf("\n---- OpenMP K-Means Results ----\n");
    printf("Threads Used: %d\n", num_threads);
    printf("Execution Time: %f seconds\n", end - start);

    printf("\nFinal Cluster Assignments:\n");
    for (int i = 0; i < n; i++) {
        printf("Point (%.2f, %.2f) -> Cluster %d\n",
               points[i].x, points[i].y, points[i].cluster);
    }

    printf("\nFinal Centroids:\n");
    for (int j = 0; j < K; j++) {
        printf("Centroid %d: (%.2f, %.2f)\n", j, centroids[j].x, centroids[j].y);
    }

    return 0;
}


Overwriting kmeans_openmp.c


In [18]:
!gcc -fopenmp -O2 kmeans_openmp.c -o kmeans_openmp -lm

In [20]:
!./kmeans_openmp 1



---- OpenMP K-Means Results ----
Threads Used: 1
Execution Time: 0.000126 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [21]:
!./kmeans_openmp 2



---- OpenMP K-Means Results ----
Threads Used: 2
Execution Time: 0.002390 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [22]:
!./kmeans_openmp 4


---- OpenMP K-Means Results ----
Threads Used: 4
Execution Time: 0.003536 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [23]:
!./kmeans_openmp 8


---- OpenMP K-Means Results ----
Threads Used: 8
Execution Time: 0.009953 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [24]:
!./kmeans_openmp 16


---- OpenMP K-Means Results ----
Threads Used: 16
Execution Time: 0.043097 seconds

Final Cluster Assignments:
Point (1.00, 2.00) -> Cluster 0
Point (2.00, 1.00) -> Cluster 0
Point (3.00, 3.00) -> Cluster 0
Point (8.00, 8.00) -> Cluster 1
Point (9.00, 8.00) -> Cluster 1
Point (8.00, 9.00) -> Cluster 1
Point (4.00, 5.00) -> Cluster 2
Point (5.00, 6.00) -> Cluster 2
Point (6.00, 5.00) -> Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)


In [35]:
%%writefile kmeans_mpi.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <mpi.h>

#define MAX_ITER 100
#define K 3   // Number of clusters

typedef struct {
    double x, y;
    int cluster;
} Point;

typedef struct {
    double x, y;
} Centroid;

// Euclidean distance
double distance(Point p, Centroid c) {
    return sqrt((p.x - c.x)*(p.x - c.x) + (p.y - c.y)*(p.y - c.y));
}

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    double start_time, end_time;

    // Dataset (can scale up for performance testing)
    int n = 9;
    Point *allPoints = NULL;

    if (rank == 0) {
        allPoints = (Point*)malloc(n * sizeof(Point));
        Point temp[9] = {
            {1,2}, {2,1}, {3,3},
            {8,8}, {9,8}, {8,9},
            {4,5}, {5,6}, {6,5}
        };
        for (int i = 0; i < n; i++) allPoints[i] = temp[i];
    }

    Centroid centroids[K] = {{2,2}, {8,8}, {5,5}};

    // Calculate number of points per process
    int base_n = n / size;
    int remainder = n % size;
    int local_n = (rank < remainder) ? base_n + 1 : base_n;

    Point *localPoints = (Point*)malloc(local_n * sizeof(Point));

    // Scatter points
    int *sendcounts = NULL;
    int *displs = NULL;

    if (rank == 0) {
        sendcounts = (int*)malloc(size * sizeof(int));
        displs = (int*)malloc(size * sizeof(int));
        int offset = 0;
        for (int i = 0; i < size; i++) {
            sendcounts[i] = (i < remainder) ? base_n + 1 : base_n;
            displs[i] = offset;
            offset += sendcounts[i];
        }
    }

    MPI_Scatterv(
        allPoints, sendcounts, displs, MPI_BYTE,
        localPoints, local_n * sizeof(Point), MPI_BYTE,
        0, MPI_COMM_WORLD
    );

    MPI_Barrier(MPI_COMM_WORLD);
    start_time = MPI_Wtime();

    // Main iteration
    for (int iter = 0; iter < MAX_ITER; iter++) {
        // Assign points to nearest cluster
        for (int i = 0; i < local_n; i++) {
            double minDist = INFINITY;
            int best = 0;
            for (int j = 0; j < K; j++) {
                double d = distance(localPoints[i], centroids[j]);
                if (d < minDist) {
                    minDist = d;
                    best = j;
                }
            }
            localPoints[i].cluster = best;
        }

        // local sums and counts
        double local_sumX[K] = {0}, local_sumY[K] = {0};
        int local_count[K] = {0};
        for (int i = 0; i < local_n; i++) {
            int c = localPoints[i].cluster;
            local_sumX[c] += localPoints[i].x;
            local_sumY[c] += localPoints[i].y;
            local_count[c]++;
        }

        // Reduce to global sums
        double global_sumX[K], global_sumY[K];
        int global_count[K];
        MPI_Allreduce(local_sumX,  global_sumX,  K, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
        MPI_Allreduce(local_sumY,  global_sumY,  K, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
        MPI_Allreduce(local_count, global_count, K, MPI_INT,    MPI_SUM, MPI_COMM_WORLD);

        // Update centroids on root
        if (rank == 0) {
            for (int j = 0; j < K; j++) {
                if (global_count[j] > 0) {
                    centroids[j].x = global_sumX[j] / global_count[j];
                    centroids[j].y = global_sumY[j] / global_count[j];
                }
            }
        }

        // Broadcast updated centroids
        MPI_Bcast(centroids, K * sizeof(Centroid), MPI_BYTE, 0, MPI_COMM_WORLD);
    }

    MPI_Barrier(MPI_COMM_WORLD);
    end_time = MPI_Wtime();

    if (rank == 0) {
        printf("\n MPI K-Means Results\n");
        printf("Processes Used: %d\n", size);
        printf("Execution Time: %f seconds\n", end_time - start_time);
        printf("\nFinal Centroids:\n");
        for (int j = 0; j < K; j++) {
            printf("Centroid %d: (%.2f, %.2f)\n", j, centroids[j].x, centroids[j].y);
        }
    }

    free(localPoints);
    if (rank == 0) {
        free(allPoints);
        free(sendcounts);
        free(displs);
    }

    MPI_Finalize();
    return 0;
}


Overwriting kmeans_mpi.c


In [36]:
!mpicc kmeans_mpi.c -o kmeans_mpi -lm


In [38]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --allow-run-as-root --oversubscribe -np 1 ./kmeans_mpi



 MPI K-Means Results
Processes Used: 1
Execution Time: 0.000044 seconds

Final Centroids:
Centroid 0: (0.11, 0.00)
Centroid 1: (8.00, 8.00)
Centroid 2: (5.00, 5.00)


In [39]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --allow-run-as-root --oversubscribe -np 2 ./kmeans_mpi


 MPI K-Means Results
Processes Used: 2
Execution Time: 0.000385 seconds

Final Centroids:
Centroid 0: (1144280264902606642906518901936106413685994075226799276395750030922870048613948610338828670840897309381417757991890339060005125589496811823373049042558183634386958059683963143898247586149042679608255770206244586335497317218300341457509599412224.00, 0.00)
Centroid 1: (8.00, 8.00)
Centroid 2: (0.00, 0.00)


In [41]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --allow-run-as-root --oversubscribe -np 4 ./kmeans_mpi


 MPI K-Means Results
Processes Used: 4
Execution Time: 0.002905 seconds

Final Centroids:
Centroid 0: (0.00, 0.00)
Centroid 1: (8.00, 8.00)
Centroid 2: (5.00, 5.00)


In [42]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --allow-run-as-root --oversubscribe -np 8 ./kmeans_mpi


 MPI K-Means Results
Processes Used: 8
Execution Time: 0.010588 seconds

Final Centroids:
Centroid 0: (0.00, 0.00)
Centroid 1: (8.00, 8.00)
Centroid 2: (5.00, 5.00)


In [43]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --allow-run-as-root --oversubscribe -np 16 ./kmeans_mpi


 MPI K-Means Results
Processes Used: 16
Execution Time: 0.070056 seconds

Final Centroids:
Centroid 0: (0.00, 0.00)
Centroid 1: (8.00, 8.00)
Centroid 2: (5.00, 5.00)


In [46]:
%%writefile kmeans_cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>
#include <time.h>

#define MAX_ITER 100
#define K 3       // no of clusters
#define N 10000   // no of points

typedef struct {
    float x, y;
    int cluster;
} Point;

typedef struct {
    float x, y;
} Centroid;

// assign points to nearest centroid(kernel)
__global__ void assignClusters(Point *points, Centroid *centroids, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= n) return;

    float minDist = 1e9;
    int cluster = 0;
    for (int j = 0; j < K; j++) {
        float dx = points[idx].x - centroids[j].x;
        float dy = points[idx].y - centroids[j].y;
        float dist = sqrtf(dx*dx + dy*dy);
        if (dist < minDist) {
            minDist = dist;
            cluster = j;
        }
    }
    points[idx].cluster = cluster;
}

// compute partial sums for centroid update
__global__ void computePartialSums(Point *points, float *sumX, float *sumY, int *count, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= n) return;

    int c = points[idx].cluster;
    atomicAdd(&sumX[c], points[idx].x);
    atomicAdd(&sumY[c], points[idx].y);
    atomicAdd(&count[c], 1);
}

int main(int argc, char *argv[]) {
    int threads = 256; // default threads (per block)
    if (argc > 1) threads = atoi(argv[1]);

    // Initialize points
    Point *h_points = (Point*) malloc(N * sizeof(Point));
    srand(42);
    for (int i = 0; i < N; i++) {
        h_points[i].x = ((float)rand() / RAND_MAX) * 10.0f;
        h_points[i].y = ((float)rand() / RAND_MAX) * 10.0f;
        h_points[i].cluster = -1;
    }

    // Initialize centroids
    Centroid h_centroids[K];
    for (int j = 0; j < K; j++) {
        h_centroids[j].x = ((float)rand() / RAND_MAX) * 10.0f;
        h_centroids[j].y = ((float)rand() / RAND_MAX) * 10.0f;
    }

    // Device memory
    Point *d_points;
    Centroid *d_centroids;
    float *d_sumX, *d_sumY;
    int *d_count;

    cudaMalloc(&d_points, N * sizeof(Point));
    cudaMalloc(&d_centroids, K * sizeof(Centroid));
    cudaMalloc(&d_sumX, K * sizeof(float));
    cudaMalloc(&d_sumY, K * sizeof(float));
    cudaMalloc(&d_count, K * sizeof(int));

    cudaMemcpy(d_points, h_points, N * sizeof(Point), cudaMemcpyHostToDevice);
    cudaMemcpy(d_centroids, h_centroids, K * sizeof(Centroid), cudaMemcpyHostToDevice);

    int blocks = (N + threads - 1) / threads;
    float start = (float)clock() / CLOCKS_PER_SEC;

    for (int iter = 0; iter < MAX_ITER; iter++) {
        // Assign clusters
        assignClusters<<<blocks, threads>>>(d_points, d_centroids, N);
        cudaDeviceSynchronize();

        // Reset sums
        cudaMemset(d_sumX, 0, K * sizeof(float));
        cudaMemset(d_sumY, 0, K * sizeof(float));
        cudaMemset(d_count, 0, K * sizeof(int));

        // Compute partial sums
        computePartialSums<<<blocks, threads>>>(d_points, d_sumX, d_sumY, d_count, N);
        cudaDeviceSynchronize();

        // Copy back to host
        float h_sumX[K], h_sumY[K];
        int h_count[K];
        cudaMemcpy(h_sumX, d_sumX, K * sizeof(float), cudaMemcpyDeviceToHost);
        cudaMemcpy(h_sumY, d_sumY, K * sizeof(float), cudaMemcpyDeviceToHost);
        cudaMemcpy(h_count, d_count, K * sizeof(int), cudaMemcpyDeviceToHost);

        // Update centroids on host
        for (int j = 0; j < K; j++) {
            if (h_count[j] > 0) {
                h_centroids[j].x = h_sumX[j] / h_count[j];
                h_centroids[j].y = h_sumY[j] / h_count[j];
            }
        }

        // Copy centroids back to device
        cudaMemcpy(d_centroids, h_centroids, K * sizeof(Centroid), cudaMemcpyHostToDevice);
    }

    float end = (float)clock() / CLOCKS_PER_SEC;

    printf("CUDA K-Means with %d threads per block completed.\n", threads);
    printf("Execution time: %f seconds\n", end - start);
    printf("Final Centroids:\n");
    for (int j = 0; j < K; j++)
        printf("Centroid %d: (%.2f, %.2f)\n", j, h_centroids[j].x, h_centroids[j].y);

    cudaFree(d_points);
    cudaFree(d_centroids);
    cudaFree(d_sumX);
    cudaFree(d_sumY);
    cudaFree(d_count);
    free(h_points);

    return 0;
}


Writing kmeans_cuda.cu


In [47]:
!nvcc kmeans_cuda.cu -o kmeans_cuda -O2



In [48]:
!./kmeans_cuda 128

CUDA K-Means with 128 threads per block completed.
Execution time: 0.013506 seconds
Final Centroids:
Centroid 0: (0.13, 1.84)
Centroid 1: (9.74, 6.66)
Centroid 2: (4.91, 7.36)


In [49]:
!./kmeans_cuda 256

CUDA K-Means with 256 threads per block completed.
Execution time: 0.015507 seconds
Final Centroids:
Centroid 0: (0.13, 1.84)
Centroid 1: (9.74, 6.66)
Centroid 2: (4.91, 7.36)


In [50]:
!./kmeans_cuda 512

CUDA K-Means with 512 threads per block completed.
Execution time: 0.015461 seconds
Final Centroids:
Centroid 0: (0.13, 1.84)
Centroid 1: (9.74, 6.66)
Centroid 2: (4.91, 7.36)


In [51]:
!./kmeans_cuda 1024

CUDA K-Means with 1024 threads per block completed.
Execution time: 0.015478 seconds
Final Centroids:
Centroid 0: (0.13, 1.84)
Centroid 1: (9.74, 6.66)
Centroid 2: (4.91, 7.36)
